In [1]:
# !pip install python-dotenv
# !pip install langchain-upstage

In [2]:
from dotenv import load_dotenv
import os
import json
import sys
import time
from tqdm import tqdm
from langchain_upstage import UpstageLayoutAnalysisLoader
from langchain_core.documents.base import Document

# Load environment variables
load_dotenv()

# Load API keys from environment variables
UPSTAGE_API_KEY = os.getenv('UPSTAGE_API_KEY')

In [3]:
def main(file_path, output_path):
    start_time = time.time()
    
    # For image files, set use_ocr to True to perform OCR inference on the document before layout detection.
    loader = UpstageLayoutAnalysisLoader(file_path, split="page", api_key=UPSTAGE_API_KEY, use_ocr=True)
    
    # For improved memory efficiency, consider using the lazy_load method to load documents page by page.
    pages = list(tqdm(loader.load(), desc="Processing pages", unit="page"))
    
    # OCR 결과물을 json으로 저장
    pages_list = [
        {"text": page.page_content, "metadata": page.metadata} for page in pages
    ]

    with open(output_path, "w", encoding="utf-8") as file: 
        json.dump(pages_list, file, ensure_ascii=False, indent=4)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"OCR processing for {file_path} completed in {elapsed_time:.2f} seconds.")

def load_documents_from_json(file_name):
    with open(file_name, "r", encoding="utf-8") as file:
        docs_list = json.load(file)
    return [
        Document(page_content=doc["text"], metadata=doc["metadata"]) for doc in docs_list
    ]

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [4]:
from glob import glob

In [5]:
reports = glob('./data/bill/*.pdf')

In [6]:
reports

['./data/bill/환자군별청구액및간병비.pdf']

In [8]:
results = ['./data/bill/환자군별청구액및간병비.json']

In [9]:
results

['./data/bill/환자군별청구액및간병비.json']

In [15]:
for report, result in zip(reports, results):
    if os.path.isfile(result):
        print(result + ': ocr 완료')
        pass
    else:
        main(report, result)

Processing pages: 100%|██████████| 1/1 [00:00<00:00, 22192.08page/s]

OCR processing for ./data/bill/환자군별청구액및간병비.pdf completed in 3.80 seconds.


In [10]:
with open(results[0]) as f:
    data = json.load(f)

In [11]:
print(data[0]['text'])

<table id='0' style='font-size:20px'><tr><td>(2024년)환자군별</td><td>공단청구금/본인부담금/간병비 산출내역</td></tr><tr><td></td><td></td></tr></table> <h1 id='1' style='font-size:16px'>1. 공단청구금</h1> <br><table id='2' style='font-size:16px'><tr><td>환 자 군</td><td>일당정액수가</td><td>필요인력가산</td><td>환자안전관리료</td><td>감염관리료</td><td>총진료비</td><td>식 대</td><td>공단청구금</td><td>본인부담금</td><td>요양급여총액</td></tr><tr><td>의료최고도</td><td>85,050</td><td rowspan="5">51,300</td><td rowspan="5">47,100</td><td rowspan="5">66,600</td><td>2,716,500</td><td rowspan="5">564,300</td><td>2,455,350</td><td>825,450</td><td>3,280,800</td></tr><tr><td>의료고도</td><td>75,510</td><td>2,430,300</td><td>2,226,390</td><td>768,210</td><td>2,994,600</td></tr><tr><td>의료중도</td><td>64,620</td><td>2,103,600</td><td>1,965,030</td><td>702,870</td><td>2,667,900</td></tr><tr><td>의료경도</td><td>62,510</td><td>2,040,300</td><td>1,914,390</td><td>690,210</td><td>2,604,600</td></tr><tr><td>선택입원군</td><td>47,000</td><td>1,575,000</td><td>1,227,150</td><td>912,150</td><td>2,

In [12]:
!pip install --upgrade --quiet  markdownify

In [4]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('../data/bill/bills.txt')
data = loader.load()

print(type(data))
print(len(data))

<class 'list'>
1


In [5]:
from dotenv import load_dotenv
import os
import json
import sys
import time
from tqdm import tqdm
# from langchain_community.vectorstores import Chroma
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_upstage import UpstageEmbeddings

# Load environment variables
load_dotenv()

# Load API keys from environment variables
UPSTAGE_API_KEY = os.getenv('UPSTAGE_API_KEY')

start_time = time.time()

# Use the recursive character splitter
recur_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
    separators=["\n","|"," "]
)

# Perform the splits using the splitter
data_splits = list(tqdm(recur_splitter.split_documents(data), desc="Splitting documents", unit="chunk"))
print(f"Number of splits: {len(data_splits)}")

# Vector Store 구축
embeddings = UpstageEmbeddings(
    api_key=UPSTAGE_API_KEY,
    model="solar-embedding-1-large"
)

persist_directory = '../.cache/db/bill'

vectordb = Chroma.from_documents(
    documents=data_splits, # 위에서 처리한 데이터 
    embedding=embeddings, # upstage solar embedding 1 large
    persist_directory=persist_directory)

vectordb.persist()
vectordb = None

end_time = time.time()
elapsed_time = end_time - start_time

Splitting documents: 100%|██████████| 2/2 [00:00<00:00, 25970.92chunk/s]


Number of splits: 2


In [3]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import RunnablePassthrough

from langchain.vectorstores import Chroma
from langchain_upstage import ChatUpstage, UpstageEmbeddings

In [6]:
llm = ChatUpstage(api_key=os.getenv("UPSTAGE_API_KEY"))


# Embeddings setup
embeddings = UpstageEmbeddings(
  api_key=os.getenv("UPSTAGE_API_KEY"),
  model="solar-embedding-1-large"
)

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

retriever = vectordb.as_retriever()

RAG_PROMPT_TEMPLATE="""

# 당신의 역할
- 당신은 요양병원에서 환자들의 보호자에게 병원비를 산출하여 알려주는 시스템입니다.
- 환자 혹은 보호자의 질문에 친절하게 대답해주세요.

------

# 병원비 참고자료 : \n\t{context}

------

# 참고사항
- 보호자가 내야하는 병원비는 공단청구금의 본인부담금과 비보험비 총액으로 계산합니다.
- 환자의 정확한 정보가 없다면 4인실, 의료중도 환자를 기준으로 병원비를 산정해주세요.

------

# 보호자의 질문 : \n\t{question}

------

# 중요
- 병원비 참고자료를 참고해주세요.
- 본인부담금과 비보험비 총액(간병비와 병실료 차액 등)을 더해서 계산하는 과정을 하나하나 설명해주세요.
- 한국어로 답변해주세요
- 답변 뒤에 '정확한 병원비는 병원에 문의해주시면 더 정확한 병원비를 알 수 있습니다.'를 추가해주세요. 

# 답변 :
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the RAG chain
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

/Users/seongyeon/Desktop/Nursing-home/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [7]:
query = "1인실에 10일 입원했다면 얼마정도 병원비가 예상될까?"
print(qa_chain.invoke(query))

1인실에 10일 입원한 경우, 병원비는 약 2,399,000원입니다.

공단청구금(공단에서 지원하는 금액)은 1인실, 의료중도 환자의 경우 1일당 85,050원이며, 10일간의 공단청구금은 850,500원입니다.

본인부담금(보호자가 직접 지불해야 하는 금액)은 공단청구금에서 의료급여 수급권자의 경우 10%를, 차상위 계층의 경우 15%를, 일반 환자의 경우 20%를 계산합니다. 따라서, 1인실, 의료중도 환자의 경우 10일간의 본인부담금은 170,100원입니다.

비보험비는 간병비와 병실료 차액으로 구성됩니다. 1:1 개인 간병비의 경우 1일당 약 130,000원이며, 10일간의 간병비는 1,300,000원입니다. 병실료 차액은 1인실의 경우 1일당 약 70,000원이며, 10일간의 병실료 차액은 700,000원입니다.

따라서, 비보험비는 간병비와 병실료 차액을 합한 2,000,000원입니다.

병원비는 공단청구금(850,500원) + 본인부담금(170,100원) + 비보험비(2,000,000원) = 2,399,000원입니다.

하지만, 정확한 병원비는 환자의 상태와 치료 방법 등에 따라 달라질 수 있으므로, 병원에 문의하여 더 정확한 병원비를 확인하시는 것이 좋습니다.
